<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/Sihyun/data_exploration_judges(judge_bias_April_20).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/School/MADS/SIADS 699 : Capstone/Team

/content/drive/MyDrive/School/MADS/SIADS 699 : Capstone/Team


Import libraries

In [3]:
import pandas as pd
import numpy as np
import altair as alt
import re
from datetime import datetime, timedelta
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns
from scipy import stats
from natsort import natsorted

In [4]:
!pip install --upgrade altair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2


In [5]:
#!ls

In [6]:
# Load all sheets into a dictionary of DataFrames
webscraping = pd.read_excel('Webscraping.xlsx', sheet_name=None)


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
webscraping.keys()


dict_keys(['Events and8', 'Events', 'Events WDSF', 'Battles Trivium', 'Battles and8 Threefold', 'Battles and8 Single Slider', 'Battles and8 Round by Round', 'Battles and8 Traditional', 'Research Questions', 'Notes', 'Notes Columns'])

In [8]:
#Load battle winners
#threefold
threefold = webscraping['Battles and8 Threefold']
threefold.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category', 'breaker 1': 'breaker 1 (red)', 'breaker 2': 'breaker 2 (blue)'}, inplace=True)
threefold[['Event', 'Category']] = threefold[['Event', 'Category']].fillna(method='ffill')
#Trivium
trivium=webscraping['Battles Trivium']
trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
trivium[['Event', 'Category']]= trivium[['Event', 'Category']].fillna(method='ffill')
#traditional
traditional = webscraping['Battles and8 Traditional']
traditional.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category', 'breaker 1': 'breaker 1 (red)', 'breaker 2': 'breaker 2 (blue)'}, inplace=True)
traditional[['Event', 'Category']] = traditional[['Event', 'Category']].fillna(method='ffill')

In [9]:
tr=traditional[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()
th=threefold[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()
tv=trivium[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()

dataframes = [('traditional', tr), ('threefold', th), ('trivium', tv)]

# Create an empty list to store modified dataframes
appended_dfs = []

# Iterate to create a combined dataframe
for name, dframe in dataframes:
    dframe['Original_DF'] = name
    dframe['Battle_ID'] = dframe.index
    appended_dfs.append(dframe)
appendedBattles = pd.concat(appended_dfs, ignore_index=True)

In [10]:
eventsAnd8=webscraping['Events and8'][1:].copy()
new_header= eventsAnd8.iloc[0]
eventsAnd8=eventsAnd8.drop(1)
eventsAnd8.columns = new_header
eventsAnd8.rename(columns={'Name':'Event'}, inplace=True)
eventsAnd8.drop_duplicates(subset=['Event'], inplace=True)

eventsWDSF = webscraping['Events WDSF'][1:].copy()
new_header = eventsWDSF.iloc[0]
eventsWDSF = eventsWDSF.drop(1)
eventsWDSF.columns = new_header
eventsWDSF.rename(columns={'Name': 'Event'}, inplace=True)
eventsWDSF.drop_duplicates(subset=['Event'], inplace=True)

Events = webscraping['Events'][1:].copy()
new_header = Events.iloc[0]
Events = Events.drop(1)
Events.columns = new_header
Events.rename(columns={'Name':'Event'}, inplace=True)
Events.drop_duplicates(subset=['Event'], inplace=True)

combined_events = pd.concat([eventsAnd8, eventsWDSF, Events], ignore_index=True)


In [11]:
#add battle date
appendedBattles=pd.merge(appendedBattles, combined_events[['Date', 'Event']], on='Event', how='left')


In [12]:
appendedBattles.tail(2)

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,Original_DF,Battle_ID,Date
2627,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Stiv,D-Matt,D-Matt,trivium,1524,2017-06-10 00:00:00
2628,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,D-Matt,Rerunn,D-Matt,trivium,1525,2017-06-10 00:00:00


Threefold Battles

In [13]:
#threefold battles
# Index threefold.at[238, 'battle rounds'] = 2 update value 2 from 4(wrong!)

threefold = webscraping['Battles and8 Threefold']
threefold.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
threefold[['Event', 'Category']] = threefold[['Event', 'Category']].fillna(method='ffill')
threefold.head(2)

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,breaker 1 vote count,...,r3j7arti,r3j7inte,r3j8over,r3j8phys,r3j8arti,r3j8inte,r3j9over,r3j9phys,r3j9arti,r3j9inte
0,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,Baruzzo,Elpablix,Baruzzo,3,3,2,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,Yoga,Faby,Faby,2,3,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Change new names
threefold.rename(columns = {"breaker 1": "breaker 1 (red)", "breaker 2": "breaker 2 (blue)"}, inplace=True)


In [15]:
threefold.columns

Index(['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner',
       'battle rounds', 'number of judges', 'breaker 1 round wins',
       'breaker 2 round wins', 'breaker 1 vote count',
       ...
       'r3j7arti', 'r3j7inte', 'r3j8over', 'r3j8phys', 'r3j8arti', 'r3j8inte',
       'r3j9over', 'r3j9phys', 'r3j9arti', 'r3j9inte'],
      dtype='object', length=128)

In [16]:
# Using DataFrame.insert() to add a column
threefold.insert(2, "Method", ['three']*len(threefold), True)

In [17]:
# Create 'Date' Column with combined_events

def Date_Added(df, combined_df):
  lst = []
  for i in range(len(df)):
    lst.append(combined_df[combined_df['Event']==df['Event'].iloc[i]]['Date'].iloc[0])


  # Using DataFrame.insert() to add a column
  df.insert(0, "Date", lst, True)
  return df


In [18]:
threefold = Date_Added(threefold, appendedBattles)

Trivium Battles

In [19]:
test_trivium=webscraping['Battles Trivium']
test_trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
test_trivium[['Event', 'Category']]= test_trivium[['Event', 'Category']].fillna(method='ffill') # fill values by Event's name

In [20]:
# Using DataFrame.insert() to add a column
test_trivium.insert(2, "Method", ['trivium']*len(test_trivium), True)

In [21]:
test_trivium = Date_Added(test_trivium, combined_events)

In [22]:
test_trivium.head(2)

,Date,Event,Category,Method,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,...,r5j8but2,r5j9over,r5j9tech,r5j9vari,r5j9perf,r5j9musi,r5j9crea,r5j9pers,r5j9but1,r5j9but2
0,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,trivium,Ami,671,671,3,9,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,trivium,Sunny,Anti,Anti,3,9,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Merge with Trivium & Threefold
combined_tv_th = pd.concat([test_trivium, threefold], ignore_index=True)

In [24]:
combined_tv_th.tail()

,Date,Event,Category,Method,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,...,r3j6inte,r3j7phys,r3j7arti,r3j7inte,r3j8phys,r3j8arti,r3j8inte,r3j9phys,r3j9arti,r3j9inte
1894,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Wigor,El Nino,Wigor,2,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1895,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Mighty Jimm,Nasty,Mighty Jimm,3,7,2,...,0.33,-3.67,-4.33,-3.00,NaN,NaN,NaN,NaN,NaN,NaN
1896,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Daniel,Mass,Mass,3,7,1,...,0.67,2.33,3.00,3.67,NaN,NaN,NaN,NaN,NaN,NaN
1897,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Quake,Flo,Quake,2,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1898,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Neosan,Lilbom,Lilbom,2,7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:

print(tabulate(combined_tv_th.tail(10), headers='keys', tablefmt='psql'))

+------+---------------------+--------------------------------------------------------+-------------------------------------+----------+-------------------+--------------------+-------------+-----------------+--------------------+------------------------+------------------------+------------------------+------------------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+----------

# Threefold -
Components :

*  'Physical Quality',
*  'Artistic Quality',
*  'Interpretive Quality'

# Trivium -
Components :

*  'Technique',
*  'Variety',
*  'Performance',
*  'Musicality',
*  'Creativity',
*  'Personality'

Create Dataframe -'Event', 'Category', 'Round number','Name', 'Overall', 'Technique', 'Variety', 'Performance', 'Musicality', 'Creativity', 'Personality'

Dictionary


*   **Date** = date of event (datetime)
*   **Event** = name of event (string)
*   **Category** = name of round (string)
*   **Round Number** = the number of each round (r1 means the first round for battle)
*   **Judge** = name of judge (string)
*   **Overall** = Sum of judge scores (float)
*   **Technique(Physical)** = judge scores for techique(trivium) or physical quality(threefold) (float)
*   **Variety(Artistic)** = judge scores for variety(trivium) or artistic quality(threefold) (float)
*   **Performance(Interpretive)** = judge scores for performance(trivium) or interpretive quality(threefold) (float)
*   **musicalirty** = judge scores for musicality (float)
*   **Creativity** = judge scores for creativity (float)
*   **Personality** = judge scores for personality (float)





# **Judge bias**:


*   Is a judge who is from the same geographic region or a member of the same crew as a competitor they are judging any more or less favorable in their voting towards that competitor?
*   Is a judge who is from the specific sex(female or male) as a competitor they are judging any more or less favorable in their voting towards that competitor?





In [26]:
event_lst=list(combined_tv_th['Event'].unique())

In [27]:
len(event_lst)

28

In [65]:
event_lst[0]

'WDSF BfG World Series - Kitakyushu 2023 - Japan'

In [28]:
first_event = combined_tv_th[combined_tv_th['Event']==event_lst[0]]

In [29]:
trivium_event = combined_tv_th[combined_tv_th['Method']=='trivium']

In [30]:
trivium_event.shape

(1526, 508)

In [62]:
combined_tv_th.shape

(1899, 508)

**We only look at Overall scores.**

if the Overall score given by judge is negative, it goes to red breaker.





In [31]:
# Keep it for the reference!

def breaker_judge_df(first_event):
  columns = ['Event', 'Category', 'breaker', 'judge', 'overall']
  new_df = pd.DataFrame(columns=columns)

  index_num = 0 # index number for new_df

  for i in range(len(first_event)):
    # print(type(test_trivium.iloc[i]))
    temp_lst = []
    lst3 = []

    num_round = first_event.iloc[i]['battle rounds'] # number of round

    num_judge = first_event.iloc[i]['number of judges'] # number of round

    num_breakers = 2
    # breaker lst
    breakers_lst = list(first_event.filter(like="breaker ").iloc[i, :num_breakers]) # breakers list
    # convert int to str!!
    if 671 in breakers_lst:
      breakers_lst[0] = str(breakers_lst[0])
      breakers_lst[1] = str(breakers_lst[1])

    for j in range(num_round):
      for k in range(num_judge):
        temp_lst.append(first_event.iloc[i]['Event'])
        temp_lst.append(first_event.iloc[i]['Category'])

        round = "r"+str(j+1) # Round number
        each_r_j_num = round+'j'+str(k+1)
        over = each_r_j_num+'over'

        overall = first_event.iloc[i][over] # Overall score

        # if overall>0, blue breaker
        temp_lst.append(np.where(overall>0, str(breakers_lst[1]), str(breakers_lst[0])))

        # append judge's name
        judge_name = 'judge '+str(k+1)+ ' name'
        temp_lst.append(first_event.iloc[i][judge_name])

        #append overall
        temp_lst.append(overall)

        # print(temp_lst)
        # print("")

        new_df.loc[index_num] = temp_lst
        index_num = index_num + 1

        temp_lst = [] # reset list

  return new_df



In [32]:
#new_df = breaker_judge_df(combined_tv_th)

new_df = breaker_judge_df(first_event)

tr_new_df = breaker_judge_df(trivium_event)

In [33]:
tr_new_df.shape

(17680, 5)

In [34]:
new_df_cp = new_df.copy()
tr_new_df_cp = tr_new_df.copy()

In [35]:
# Avoid negative
new_df_cp['overall'] = new_df_cp['overall'].abs()
tr_new_df_cp['overall'] = tr_new_df_cp['overall'].abs()

In [36]:
len(list(tr_new_df_cp['breaker'].apply(lambda x: str(x)).unique()))

426

In [37]:
print(tabulate(new_df_cp.head(), headers='keys', tablefmt='psql'))

+----+-------------------------------------------------+------------------------+-----------+-----------+-----------+
|    | Event                                           | Category               | breaker   | judge     |   overall |
|----+-------------------------------------------------+------------------------+-----------+-----------+-----------|
|  0 | WDSF BfG World Series - Kitakyushu 2023 - Japan | B-Girls Top 8 Knockout | 671       | Kujo      |     14.93 |
|  1 | WDSF BfG World Series - Kitakyushu 2023 - Japan | B-Girls Top 8 Knockout | Ami       | Lazylegz  |     10.67 |
|  2 | WDSF BfG World Series - Kitakyushu 2023 - Japan | B-Girls Top 8 Knockout | Ami       | Valentine |      0.4  |
|  3 | WDSF BfG World Series - Kitakyushu 2023 - Japan | B-Girls Top 8 Knockout | 671       | Spin      |      6.4  |
|  4 | WDSF BfG World Series - Kitakyushu 2023 - Japan | B-Girls Top 8 Knockout | Ami       | Octopus   |      4.47 |
+----+-------------------------------------------------+

In [39]:
# Load all sheets into a dictionary of DataFrames
nations_df = pd.read_excel('Nationalities.xlsx', sheet_name=None)

In [40]:
nations_df.keys()

dict_keys(['General', 'Events', 'Judges', 'Breakers', 'Breakers(first)', 'Sheet5', 'Judege_N', 'Judges(first)', 'Battles Trivium'])

In [41]:
#Load breakers nationalities for only first event
nations_breakers = nations_df['Breakers(first)']

In [42]:
nations_breakers

,Breaker,Nationality
0,671,China
1,Alessandrina,Italy
2,Ami,Japan
3,Amir,Kazakhstan
4,Anna,Denmark
...,...,...
78,Wing,Republic of Korea
79,X-Rain,China
80,Xak,Spain
81,Yell,Republic of Korea


In [43]:
#Load breakers nationalities for only first event
nations_judges = nations_df['Judges(first)']

In [44]:
nations_judges

,Judge,Nationality
0,Kujo,United States
1,Lazylegz,Canada
2,Rush,New Zealand
3,Valentine,France
4,Spin,United Kingdom
5,Kim,China
6,Aya,Chinese Taipei
7,Octopus,Republic of Korea
8,Max,Portugal
9,Katsu One,Japan


# Append Nationality - same or different

In [45]:
def create_nationality(df, breakers_lst):
  # if breaker is Sunny, nationality is same
  nat_lst = []
  for i in range(len(df)):
    if df['breaker'].iloc[i]in breakers_lst:
      nat_lst.append('same')
    else:
      nat_lst.append('different')

  # new_df0['nationality'] = nat_lst #pd.to_datetime(nat_lst)
  e = pd.Series(nat_lst)
  df = df.assign(nationality=e.values)

  return df

# Altair Chart

In [46]:
def create_chart_bias(df, nation):
  title = alt.TitleParams('Judge '+df['judge'].iloc[0]+' - '+nation, anchor='middle')
  chart = alt.Chart(df, title=title).mark_circle().encode(
      x='breaker:N',
      y='overall:Q',
      color = alt.Color('nationality')
  )
  return chart

# T-Test ( Same v.s. Different )

In [127]:
# Import the necessary libraries:


def ttest_df(df, alpha):
  # Filter the dataset for the two(same & different)  we want to compare:
  same_df = df[df['nationality'] == 'same']
  different_df = df[df['nationality'] == 'different']

  # Extract the petal lengths for each species:
  same_df_overall = same_df['overall']
  different_df_overall = different_df['overall']

  # Perform the t-test:
  t_stat, p_value = stats.ttest_ind(same_df_overall, different_df_overall)

  print('Statistics(t-value) : {} and p-value : {}'.format(t_stat, p_value))
  print('')

  # Interpret the results:
  alpha = alpha
  if p_value < alpha:
    print("Reject the null hypothesis; there is a significant difference between the same nationality and different one.")
    if t_stat < 0:
      print("{} shows bias towards different nationalities. ".format(df['judge'].iloc[0]))
    else :
      print("{} shows bias towards favorite on the same nationality. ".format(df['judge'].iloc[0]))
  else:
    print("Fail to reject the null hypothesis; there is no significant difference between between the same nationality and different one.")

  print('')

Call judge's nationality

In [48]:
def call_judge_nation(judge):
  return nations_judges[nations_judges['Judge']==judge]['Nationality'].iloc[0]


list for breakers'name by judge's nationality

In [49]:
def call_breaker_nation(j_nation):
  return list(nations_breakers[nations_breakers['Nationality']==j_nation]['Breaker'])

# Call Everything

In [87]:
def enter_judge_to_everything(df):
  print("""Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):""")
  judge = input()
  print('Hello, Judge ' + judge)
  print('')

  j_nation = call_judge_nation(judge)

  breakers_lst = call_breaker_nation(j_nation)

  new_df = df[df['judge']==judge]

  new_df = create_nationality(new_df, breakers_lst)

  ttest_df(new_df, 0.05)

  chart = create_chart_bias(new_df, j_nation)

  return chart





# Judges' Names


---

'Kujo', 'Lazylegz', 'Valentine', 'Spin', 'Octopus', 'Max', 'Aya',
       'Kim', 'Rush', 'Katsu One'

In [75]:
nations_judges['Judge']

0         Kujo
1     Lazylegz
2         Rush
3    Valentine
4         Spin
5          Kim
6          Aya
7      Octopus
8          Max
9    Katsu One
Name: Judge, dtype: object

In [95]:
enter_judge_to_everything(new_df_cp)

Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):
Kujo
Hello, Judge Kujo

Statistics(t-value) : -1.0351924342535759 and p-value : 0.30259367208597826

Fail to reject the null hypothesis; there is no significant difference between between the same nationality and different one.



alt.Chart(...)

In [96]:
enter_judge_to_everything(new_df_cp)

Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):
Lazylegz
Hello, Judge Lazylegz

Statistics(t-value) : 0.31351475073715324 and p-value : 0.7542335283304412

Fail to reject the null hypothesis; there is no significant difference between between the same nationality and different one.



alt.Chart(...)

In [97]:
enter_judge_to_everything(new_df_cp)

Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):
Rush
Hello, Judge Rush

Statistics(t-value) : nan and p-value : nan

Fail to reject the null hypothesis; there is no significant difference between between the same nationality and different one.



alt.Chart(...)

In [98]:
enter_judge_to_everything(new_df_cp)

Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):
Valentine
Hello, Judge Valentine

Statistics(t-value) : -1.6182046368986578 and p-value : 0.1081594147707067

Fail to reject the null hypothesis; there is no significant difference between between the same nationality and different one.



alt.Chart(...)

In [99]:
enter_judge_to_everything(new_df_cp)

Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):
Spin
Hello, Judge Spin

Statistics(t-value) : nan and p-value : nan

Fail to reject the null hypothesis; there is no significant difference between between the same nationality and different one.



alt.Chart(...)

In [100]:
enter_judge_to_everything(new_df_cp)

Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):
Kim
Hello, Judge Kim

Statistics(t-value) : -1.2325768123291538 and p-value : 0.22006554292194702

Fail to reject the null hypothesis; there is no significant difference between between the same nationality and different one.



alt.Chart(...)

In [101]:
enter_judge_to_everything(new_df_cp)

Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):
Aya
Hello, Judge Aya

Statistics(t-value) : nan and p-value : nan

Fail to reject the null hypothesis; there is no significant difference between between the same nationality and different one.



alt.Chart(...)

In [102]:
enter_judge_to_everything(new_df_cp)

Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):
Octopus
Hello, Judge Octopus

Statistics(t-value) : -2.291394654381566 and p-value : 0.023037546222835697

Reject the null hypothesis; there is a significant difference between the same nationality and different one.
Octopus shows bias towards different nationalities. 



alt.Chart(...)

In [103]:
enter_judge_to_everything(new_df_cp)

Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):
Max
Hello, Judge Max

Statistics(t-value) : nan and p-value : nan

Fail to reject the null hypothesis; there is no significant difference between between the same nationality and different one.



alt.Chart(...)

In [104]:
enter_judge_to_everything(new_df_cp)

Enter Judge's name ( Kujo, Lazylegz, Rush, Valentine, Spin, Kim, Aya, Octopus, Max, Katsu One):
Katsu One
Hello, Judge Katsu One

Statistics(t-value) : 3.2479051412794204 and p-value : 0.0017077633051357935

Reject the null hypothesis; there is a significant difference between the same nationality and different one.
Katsu One shows bias towards favorite on the same nationality. 



alt.Chart(...)

# Gender Bias

---
Given a judge name, we will take a t-test on overall scores between female and male breaking.


In [52]:
columns = ['Date', 'Event', 'Category','Method', 'Gender', 'Round_Number','Judge', 'Winner_In_Round', 'Overall', 'Technique_Physical', 'Variety_Artistic', 'Performance_Interpretive', 'Musicality', 'Creativity', 'Personality']

judge_df = pd.DataFrame(columns=columns)


In [53]:
# def create_judge_df(df):


event_lst = list(combined_tv_th['Event'].unique())

total_judges_lst = [] # Save the entire judges

total_breakers_lst = [] # Save the entire judges

# If there is any change, you must run dataframe judge_df first!
# Index number is initialized
index_num = 0

for event in event_lst:
  temp_lst = [] # empty list
  temp_df = combined_tv_th[combined_tv_th['Event']==event] # Take dataframe by each event

  date = temp_df['Date'].iloc[0]

  category_lst = temp_df['Category'].unique()
  #print(category_lst)

  # string of numbers by Descending order
  category_lst = natsorted(category_lst, reverse=True)

  for category in category_lst:

    cat_temp_df = temp_df[temp_df['Category']==category] # Take dataframe by category in each event
    # print(cat_temp_df['battle rounds'].unique())
    # print('')

    if "Girls" in category :
      gender = "Female"
    else:
      gender = "Male"


    for k in range(len(cat_temp_df)):
      num_judges = cat_temp_df['number of judges'].iloc[k]
      judges_lst = list(cat_temp_df.filter(like="judge ").iloc[k, :num_judges]) # Judges list
      num_round = cat_temp_df['battle rounds'].iloc[k] # number of round

      total_judges_lst = list(set(total_judges_lst + judges_lst))

      num_breakers = 2 # number of breakers
      breakers_lst = list(cat_temp_df.filter(like="breaker ").iloc[k, :num_breakers]) # breakers list

      # convert int to str!!
      if 671 in breakers_lst:
        breakers_lst[0] = str(breakers_lst[0])
        breakers_lst[1] = str(breakers_lst[1])

      total_breakers_lst = list(set(total_breakers_lst + breakers_lst))

      for i in range(num_round):
        round = "r"+str(i+1) # Round number

        for j in range(num_judges):
          temp_lst.append(date)
          temp_lst.append(event)
          temp_lst.append(category)

          method = cat_temp_df['Method'].iloc[k]
          temp_lst.append(method)

          temp_lst.append(gender)


          temp_lst.append(round)

          temp_lst.append(judges_lst[j])

          each_r_j_num = round+'j'+str(j+1)
          over = each_r_j_num+'over'

          # if trivium data
          if method =='trivium':
            # Score list - Overall, Tech, Variety, Performance, Musicality, Creative, Personality
            tech = each_r_j_num +'tech'
            vari = each_r_j_num +'vari'
            perf = each_r_j_num +'perf'
            musi = each_r_j_num +'musi'
            crea = each_r_j_num +'crea'
            pers = each_r_j_num +'pers'

            a_l = [over, tech, vari, perf, musi, crea, pers]
            s_lst = list(cat_temp_df[a_l].iloc[k])

          else:
            s_lst = [] # Save scores(by judges)

            phys = each_r_j_num +'phys'
            arti = each_r_j_num +'arti'
            inte = each_r_j_num +'inte'

            a_l = [over, phys, arti, inte]

            s_lst = list(cat_temp_df[a_l].iloc[k]) + [np.nan]*3

          # Append winner breaker by Overall score
          # if Overall > 0, blue(breaker2)
          temp_lst.append(np.where(s_lst[0]>0, str(breakers_lst[1]), str(breakers_lst[0])))

          judge_df.loc[index_num] = temp_lst+ s_lst
          index_num = index_num + 1

          temp_lst = [] # reset the list


  # return total_breakers_lst, total_judges_lst, judge_df


In [54]:

def abs_df(df):

  # Overall > 0, blue
  e0 = pd.Series(np.where(df['Overall']>0, 'blue', 'red'))
  df = df.assign(color=e0.values)

  e = df['Overall'].abs()
  df = df.assign(Overall=e.values)

  e1 = df['Technique_Physical'].abs()
  df = df.assign(Technique_Physical=e1.values)

  e2 = df['Variety_Artistic'].abs()
  df= df.assign(Variety_Artistic=e2.values)

  e3 = df['Performance_Interpretive'].abs()
  df = df.assign(Performance_Interpretive=e3.values)

  e4 = df['Musicality'].abs()
  df = df.assign(Musicality=e4.values)

  e5 = df['Creativity'].abs()
  df = df.assign(Creativity=e5.values)

  e6 = df['Personality'].abs()
  df = df.assign(Personality=e6.values)

  return df

In [156]:
# Import the necessary libraries:


def ttest_gender_df(df, alpha):
  # Filter the dataset for the two(same & different)  we want to compare:
  fema_df = df[df['Gender'] == 'Female']
  male_df = df[df['Gender'] == 'Male']

  bias_girl = 0
  bias_boy = 0

  if fema_df.shape[0] == 0:
    print("Judge {} Only score male breakers.".format(df['Judge'].iloc[0]))
    print('')
    return (bias_girl, bias_boy)

  elif male_df.shape[0] == 0:
    print("Judge {} Only score female breakers.".format(df['Judge'].iloc[0]))
    print('')
    return (bias_girl, bias_boy)

  else:

    # Extract the petal lengths for each species:
    fema_df_overall = fema_df['Overall']
    male_df_overall = male_df['Overall']

    # Perform the t-test:
    t_stat, p_value = stats.ttest_ind(fema_df_overall, male_df_overall)

    print('Statistics(t-value) : {} and p-value : {}'.format(t_stat, p_value))
    print('')

    # Interpret the results:
    alpha = alpha
    if p_value < alpha:
      print("Reject the null hypothesis; there is a significant difference between B-Girls and B-Boys.")
      if t_stat < 0:
        print("{} shows bias towards B-Girls. ".format(df['Judge'].iloc[0]))
        bias_girl =1

      else :
        print("{} shows bias towards B-Boys. ".format(df['Judge'].iloc[0]))
        bias_boy = 1
    else:
      print("Fail to reject the null hypothesis; there is no significant difference between between B-Girls and B-Boys.")

    print('')
    return (bias_girl, bias_boy)

In [157]:
ttest_gender_df(new_df, 0.05)

Judge Shie Chan Only score male breakers.



(0, 0)

In [56]:
def create_chart_gender_bias(df):
  title = alt.TitleParams('Judge '+df['Judge'].iloc[0], anchor='middle')
  chart = alt.Chart(df, title=title).mark_circle().encode(
      x='Winner_In_Round:N',
      y='Overall:Q',
      color = alt.Color('Gender')
  )
  return chart

In [57]:
judge_df[judge_df['Judge']=='Dom-K']['Winner_In_Round']

1546             Vanessa
1556               Nadia
1566               Jilou
1576        Alicia Anais
1586                  Bo
              ...       
18126    Formica Atomica
18129              Slavi
18132                Ant
18135    Formica Atomica
18138                Cat
Name: Winner_In_Round, Length: 319, dtype: object

In [158]:
def enter_judge_to_gendereverything(df):
  print("""Enter Judge's name:""")
  judge = input()
  print('Hello, Judge ' + judge)
  print('')

  new_df = df[df['Judge']==judge]

  # print(tabulate(new_df, headers='keys', tablefmt='psql'))

  ttest_gender_df(new_df, 0.05)

  chart = create_chart_gender_bias(new_df)

  return chart


In [63]:
len(total_judges_lst)

80

In [64]:
len(total_breakers_lst)

738

In [116]:
columns1 = ['Judge']

total_judge_df = pd.DataFrame(columns=columns1)


In [107]:
total_judge_df['Judge']=total_judges_lst

In [108]:
total_judge_df

,Judge
0,Flash
1,Renegade
2,Dragon Lee
3,Yang
4,Mary
...,...
75,Rush
76,Jeskilz
77,Kujo
78,Jess


In [112]:
trans_total_judge_df = total_judge_df.transpose()

trans_total_judge_df.iloc[0, :10]

0         Flash
1      Renegade
2    Dragon Lee
3          Yang
4          Mary
5          Hefo
6           Aya
7     Valentine
8    Crazy Legs
9     Movie One
Name: Judge, dtype: object

In [60]:
# take abolute values in each component
judge_df_cp = judge_df.copy()

judge_df_cp = abs_df(judge_df_cp)

In [159]:

for i in range(8):
  #print(i)
  temp_total_judge_df = pd.DataFrame(columns=columns1)
  temp_total_judge_df['Judge'] = trans_total_judge_df.iloc[0, 10*i:10+10*i]
  print(tabulate(temp_total_judge_df.transpose(), headers='keys', tablefmt='psql'))

enter_judge_to_gendereverything(judge_df_cp)

+-------+-------+----------+------------+------+------+------+-----+-----------+------------+-----------+
|       | 0     | 1        | 2          | 3    | 4    | 5    | 6   | 7         | 8          | 9         |
|-------+-------+----------+------------+------+------+------+-----+-----------+------------+-----------|
| Judge | Flash | Renegade | Dragon Lee | Yang | Mary | Hefo | Aya | Valentine | Crazy Legs | Movie One |
+-------+-------+----------+------------+------+------+------+-----+-----------+------------+-----------+
+-------+---------+-----------+------+------------+----------+-------+--------+-----------+-------+------+
|       | 10      | 11        | 12   | 13         | 14       | 15    | 16     | 17        | 18    | 19   |
|-------+---------+-----------+------+------------+----------+-------+--------+-----------+-------+------|
| Judge | RoxRite | OG Samson | Nori | Kazuki Roc | My-Key77 | Dario | Thesis | Lildragon | Steve | Feel |
+-------+---------+-----------+------+----

alt.Chart(...)

In [120]:

for i in range(8):
  #print(i)
  temp_total_judge_df = pd.DataFrame(columns=columns1)
  temp_total_judge_df['Judge'] = trans_total_judge_df.iloc[0, 10*i:10+10*i]
  print(tabulate(temp_total_judge_df.transpose(), headers='keys', tablefmt='psql'))

enter_judge_to_gendereverything(judge_df_cp)

+-------+-------+----------+------------+------+------+------+-----+-----------+------------+-----------+
|       | 0     | 1        | 2          | 3    | 4    | 5    | 6   | 7         | 8          | 9         |
|-------+-------+----------+------------+------+------+------+-----+-----------+------------+-----------|
| Judge | Flash | Renegade | Dragon Lee | Yang | Mary | Hefo | Aya | Valentine | Crazy Legs | Movie One |
+-------+-------+----------+------------+------+------+------+-----+-----------+------------+-----------+
+-------+---------+-----------+------+------------+----------+-------+--------+-----------+-------+------+
|       | 10      | 11        | 12   | 13         | 14       | 15    | 16     | 17        | 18    | 19   |
|-------+---------+-----------+------+------------+----------+-------+--------+-----------+-------+------|
| Judge | RoxRite | OG Samson | Nori | Kazuki Roc | My-Key77 | Dario | Thesis | Lildragon | Steve | Feel |
+-------+---------+-----------+------+----

alt.Chart(...)

In [121]:

for i in range(8):
  #print(i)
  temp_total_judge_df = pd.DataFrame(columns=columns1)
  temp_total_judge_df['Judge'] = trans_total_judge_df.iloc[0, 10*i:10+10*i]
  print(tabulate(temp_total_judge_df.transpose(), headers='keys', tablefmt='psql'))

enter_judge_to_gendereverything(judge_df_cp)

+-------+-------+----------+------------+------+------+------+-----+-----------+------------+-----------+
|       | 0     | 1        | 2          | 3    | 4    | 5    | 6   | 7         | 8          | 9         |
|-------+-------+----------+------------+------+------+------+-----+-----------+------------+-----------|
| Judge | Flash | Renegade | Dragon Lee | Yang | Mary | Hefo | Aya | Valentine | Crazy Legs | Movie One |
+-------+-------+----------+------------+------+------+------+-----+-----------+------------+-----------+
+-------+---------+-----------+------+------------+----------+-------+--------+-----------+-------+------+
|       | 10      | 11        | 12   | 13         | 14       | 15    | 16     | 17        | 18    | 19   |
|-------+---------+-----------+------+------------+----------+-------+--------+-----------+-------+------|
| Judge | RoxRite | OG Samson | Nori | Kazuki Roc | My-Key77 | Dario | Thesis | Lildragon | Steve | Feel |
+-------+---------+-----------+------+----

alt.Chart(...)

As you can see that you can pick any judge name you wanted.


In [167]:

# Create a count dataframe for each round
columns = ['Total', 'Total Bias', 'Total Bias Girl', 'Total Bias Boy']
bias_df = pd.DataFrame(columns=columns)

total_bias = 0
total_bias_girl = 0
total_bias_boy = 0
bias_lst = []
total = len(total_judges_lst)
bias_lst.append(total)

for judge in total_judges_lst:
  #print(judge)

  new_df = judge_df_cp[judge_df_cp['Judge']==judge]

  g, b = ttest_gender_df(new_df, 0.05)

  total_bias = total_bias + g + b
  total_bias_girl = total_bias_girl + g
  total_bias_boy = total_bias_boy +b

bias_lst.append(total_bias)
bias_lst.append(total_bias_girl)
bias_lst.append(total_bias_boy)

bias_df.loc[0] = bias_lst
print("Total bias : {}, Total bias percentage : {}".format(total_bias, total_bias/total))
print("Total bias girl: {}, Total bias girl percentage : {}".format(total_bias_girl, total_bias_girl/total))
print("Total bias boy: {}, Total bias boy percentage : {}".format(total_bias_boy, total_bias_boy/total))

Judge Flash Only score male breakers.

Statistics(t-value) : -2.520409145451641 and p-value : 0.01186680630343862

Reject the null hypothesis; there is a significant difference between B-Girls and B-Boys.
Renegade shows bias towards B-Girls. 

Statistics(t-value) : 0.018020685655257166 and p-value : 0.9856970509784924

Fail to reject the null hypothesis; there is no significant difference between between B-Girls and B-Boys.

Statistics(t-value) : 2.837247915126003 and p-value : 0.004960330460289512

Reject the null hypothesis; there is a significant difference between B-Girls and B-Boys.
Yang shows bias towards B-Boys. 

Statistics(t-value) : -0.8855747598458062 and p-value : 0.3762302404774054

Fail to reject the null hypothesis; there is no significant difference between between B-Girls and B-Boys.

Statistics(t-value) : -1.0912161505286468 and p-value : 0.275735677670063

Fail to reject the null hypothesis; there is no significant difference between between B-Girls and B-Boys.

Stat

In [162]:
bias_df

,Total,Total Bias,Total Bias Girl,Total Bias Boy
0,80,18,10,8


In [165]:
bias_melt = bias_df.melt()

In [168]:
alt.Chart(bias_melt).mark_bar().encode(
    x='variable',
    y='value',
    color = 'variable'
)

alt.Chart(...)